## Imports

In [3]:
import numpy as np
import cv2
import csv
import glob
import os
import pandas as pd
import time


from IPython.display import clear_output

def update_progress(progress):
    bar_length = 60
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### **Preprocessing**
##Iterate over frames in each video to:
1. Crop the face using haarcascade face detector.
2. Calculate flow between each two consecutive frames
3. Save the cropped image and the flow image.

In [1]:
# Uncomment following when running on Drive
'''
from google.colab import drive
drive.mount('/content/drive')
home_dir = '/content/drive/My Drive/Colab Notebooks/FER/'
baum_dir = home_dir + 'BAUM/'
'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Uncomment following when running on local device
# Add Home dir upto BAUM folder
home_dir = '..\\'#TODO add you home directory
baum_dir = home_dir + 'BAUM\\'

In [6]:
src_folder_name = 'BAUM1s_MP4 - All'
dst_folder_spatial = 'imgs_spatial'
dst_folder_flow = 'imgs_flow'
src_csv_name = "Annotations_BAUM1s.xlsx"
target_codes = [1, 6, 7, 8, 10, 11] # for 1s


In [7]:
videos = glob.glob(baum_dir + src_folder_name + '/*/*.mp4')
videos = sorted(videos)
print (len(videos))

1198


In [16]:
temp_image = np.array([])
none_image = None

data_labels_xl =  pd.read_excel(baum_dir + src_csv_name)

data_labels = np.array(data_labels_xl)

videos = glob.glob(baum_dir + src_folder_name + '/*/*.mp4')
videos = sorted(videos)
print(videos[0], len(videos), " videos was detected in baum_dir")


# Create a face detec tion module 
calssifiers =['haarcascade_frontalface_default.xml',
              'haarcascade_frontalface_alt2.xml',
              'haarcascade_frontalcatface_extended.xml',
              'haarcascade_frontalcatface.xml',
              'haarcascade_frontalface_alt_tree.xml',
              'haarcascade_frontalface_alt.xml']

def detectFace(img):
    for classifier in calssifiers:
        face_cascade = cv2.CascadeClassifier(home_dir +'face_detectors/' + classifier)
        faces = face_cascade.detectMultiScale(img, 1.3, 1)
        if (len(faces) > 0):
            return faces
    faces = []
    return faces

../BAUM/BAUM1s_MP4 - All\s001\S001_005.mp4 1198  videos was detected in baum_dir


In [19]:
data_generated = 0
if not data_generated:
  videoSkipped = 0
  allbadFrames_count = 0
  videoProcessed = 0
  for video in videos:
      #print(video)
      # Test if this video has been already procedded
      already_processed = 0
      flow_path = video.replace(src_folder_name, dst_folder_flow).replace('.mp4', '/')
      spatial_path = video.replace(src_folder_name, dst_folder_spatial).replace('.mp4', '/')
      flow_relative_path = flow_path.split('/')[-4] + '/' + flow_path.split('/')[-3] + '/' + flow_path.split('/')[-2] 
      spatial_relative_path = spatial_path.split('/')[-4] + '/' + spatial_path.split('/')[-3] + '/' + spatial_path.split('/')[-2]
      folder = spatial_relative_path
      #print(folder)
      # Test if this video in one of the 6 classes
      sample_name = video.split('/')[-1].split('\\')[-1].split('.')[0]
      print (sample_name)
      label_found = 0
      for label in data_labels:
        if label[3] == sample_name:
            emotion = label[4]
            code = label[5]
            if code in target_codes:
              label_found = 1
              #print("In labels")
              break

      if (not label_found) or (already_processed):
        pass #print("already processed or out of classes")
      else:
        #print(video)
        videoProcessed += 1
        cap = cv2.VideoCapture(video)
        no_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
        ret, old_frame = cap.read()
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

        faces = detectFace(old_gray)
        count = 0
        while (len(faces) == 0):    
            if count > no_of_frames - 1: #Stop looking for a valid frame at the end of the video #
                break
            count+=1
            #print("First Frame is not good for face detection ", count, no_of_frames)
            ret, old_frame = cap.read()
            if isinstance(old_frame, type(temp_image)) and not isinstance(old_frame, type(none_image)) :
                old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
                faces = detectFace(old_gray)
            else:
                #print ("bad frame", type(old_frame), video)
                pass 
                
                
                      
        if (len(faces) == 0):
            #print ("All Video Frames are BAD", video)
            videoSkipped += 1
            continue
        
        (x,y,w,h) = faces[0]
        old_gray = old_gray[y:y+h, x:x+w]
        old_color = old_frame[y:y+h, x:x+w]
        size_gray = old_gray.shape
        size_color = old_color.shape

        # Create a mask image for drawing purposes
        mask = np.zeros_like(old_color)
        hsv = np.zeros_like(old_color)
        hsv[...,1] = 255
        
        noFacesCnt = 0
        calcOpticalFlow = 0
        for i in range(no_of_frames - 1):
            ret,frame = cap.read()
            if isinstance(frame, type(temp_image)) and not isinstance(frame, type(none_image)) :
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                faces = detectFace(frame_gray)
                if len(faces) == 0:
                    calcOpticalFlow = 0
                    noFacesCnt += 1
                    allbadFrames_count +=1
                    #print ("No Faces Found ", noFacesCnt, no_of_frames, video)
                    continue
                
                (x,y,w,h)  = faces[0]
                frame_gray = frame_gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                
                if(frame_gray.shape[0] < 90 or roi_color.shape[0] < 90 ):
                    calcOpticalFlow = 0
                    allbadFrames_count +=1
                    #print("Skip this frame , small image!! ", video , i)
                    continue
                                
                #print(frame_gray.shape)
                frame_gray = cv2.resize(frame_gray, (size_gray[1],size_gray[0]))
                roi_color = cv2.resize(roi_color, (size_color[1],size_color[0]))
                #print(frame_gray.shape, "after")
              
                calcOpticalFlow += 1
                            
                if (calcOpticalFlow > 1):
                    flow = cv2.calcOpticalFlowFarneback(old_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)            
                    mag, ang   = cv2.cartToPolar(flow[...,0], flow[...,1])
                    hsv[...,0] = ang*180/np.pi/2
                    hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
                    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
                    
                    str_i = '{:03}'.format(i) # max 999 frames

                    flow_relative_name = flow_relative_path + '/' + str_i + '.png'
                    spatial_relative_name = spatial_relative_path + '/' + str_i + '.png'
                    
                    flow_name_abs = baum_dir + flow_relative_name
                    spatial_name_abs = baum_dir + spatial_relative_name

                    flow_folder = flow_name_abs.replace('/'+str_i+'.png', '')
                    spatial_folder = spatial_name_abs.replace('/'+str_i+'.png', '')

                    if not os.path.isdir(flow_folder):
                        os.makedirs(flow_folder)
                    if not os.path.isdir(spatial_folder):
                        #print("create folder: ", spatial_folder)
                        os.makedirs(spatial_folder)
                    
                    cv2.imwrite(flow_name_abs, rgb)
                    cv2.imwrite(spatial_name_abs,roi_color)
   
                # Now update the previous frame and previous points
                old_gray = frame_gray.copy()
              
  print("No of skipped videos is: ", videoSkipped)
  print("No of allbadFrames is: ", allbadFrames_count)
  print("No of video processed is ", videoProcessed)

S001_005
S001_006
S001_007
S001_008
S001_009
S001_010
S001_011
S001_012
S001_013
S001_014
S001_015
S002_001
S002_010
S002_011
S002_012
S002_013
S002_014
S002_015
S002_016
S002_017
S002_018
S002_019
S002_020
S002_021
S002_022
S002_023
S002_024
S002_025
S002_026
S002_027
S002_028
S002_029
S002_030
S002_033
S002_035
S002_036
S002_037
S002_038
S002_039
S002_040
S002_041
S002_042
S002_043
S002_045
S002_046
S002_047
S002_048
S002_049
S003_001
S003_010
S003_011
S003_012
S003_013
S003_014
S003_015
S003_016
S003_017
S003_018
S003_019
S003_020
S003_021
S003_022
S003_025
S003_027
S003_028
S003_029
S003_030
S003_031
S003_032
S003_033
S003_035
S003_036
S003_037
S003_038
S003_039
S003_040
S003_041
S003_042
S003_043
S003_044
S003_046
S003_047
S003_048
S004_001
S004_010
S004_011
S004_012
S004_013
S004_014
S004_015
S004_016
S004_017
S004_018
S004_019
S004_020
S004_021
S004_022
S004_023
S004_024
S004_025
S004_026
S004_027
S004_028
S004_029
S004_030
S004_031
S004_032
S004_033
S004_034
S004_035
S004_036
S

S024_030
S024_031
S024_033
S024_034
S024_035
S024_036
S024_037
S024_038
S024_039
S024_040
S024_041
S024_042
S025_011
S025_012
S025_013
S025_014
S025_015
S025_016
S025_017
S025_018
S025_019
S025_020
S025_021
S025_022
S025_023
S025_024
S025_025
S025_026
S025_027
S025_028
S025_029
S025_030
S025_031
S025_032
S025_033
S025_034
S025_035
S025_036
S025_037
S025_038
S025_039
S025_040
S025_041
S025_042
S025_043
S025_044
S025_045
S025_046
S025_047
S025_048
S025_049
S025_050
S026_001
S026_003
S026_004
S026_005
S026_009
S026_013
S026_014
S026_017
S026_021
S026_031
S026_032
S026_033
S026_034
S026_035
S026_036
S026_037
S026_038
S026_039
S026_040
S026_041
S026_042
S026_043
S026_044
S026_045
S026_046
S026_047
S026_048
S026_049
S026_050
S026_052
S026_053
S026_054
S026_055
S026_056
S026_057
S026_058
S026_059
S026_060
S026_061
S026_062
S026_063
S026_064
S026_065
S026_066
S026_067
S026_068
S026_069
S026_070
S026_071
S026_072
S026_073
S026_074
S026_075
S026_076
S026_077
S026_078
S026_079
S026_080
S026_081
S

# Get All Written Images

In [57]:
from glob import glob
import os, time
import pandas as pd



csv_file_name = 'data_6_classes_1s.csv'

data_labels_xl =  pd.read_excel(baum_dir + src_csv_name)

csv_file_name = baum_dir + csv_file_name
data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(csv_file_name, mode='w', header=False, index = False, sep = ',')

print ("Target scv file is: ", csv_file_name)

flow = baum_dir + dst_folder_flow +'\\'
spatial = baum_dir + dst_folder_spatial + '\\'
print (flow)

flowImgs = glob(flow+'*/*/*.png')
count = 0
for img in flowImgs:
    flowImg = img
    spatialImg = img.replace(dst_folder_flow, dst_folder_spatial)
    if (os.path.isfile(spatialImg)):
        count += 1
        update_progress(float(count)/len(flowImgs))
        vidName = img.split('\\')[-2]
        vidLabel = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion'].values[0])
        vidCode = str(data_labels_xl.loc[data_labels_xl['Clip Name'] == vidName]['Emotion Code'].values[0])
        if (int(vidCode) in target_codes):
            #print ('Needed Emotion')
            #print (flowImg, spatialImg, vidLabel, vidCode)
            data_frame =  pd.DataFrame([[flowImg, spatialImg,  vidLabel, vidCode]])
            data_frame.to_csv(csv_file_name, mode='a', header=False, index = False, sep = ',')

print ("Total found images: ", len(flowImgs), count)

Progress: [############################################################] 100.0%
Total found images:  54410 54410


### **Splitting data to train / valid**

In [8]:
import pandas as pd
import numpy as np
baum_dir = home_dir + 'BAUM\\'

csv_file = 'data_6_classes_1s.csv'
data = pd.read_csv(baum_dir + csv_file )
data.sort_values(by=['code'])
data = np.array(data)

training_data_file = baum_dir + "training_data_1s.csv"
validation_data_file = baum_dir + "validation_data_1s.csv"
target_codes = [1, 6, 7, 8, 10, 11] # for 1s
#anger , disgust , Fear , Happiness, Sadness, Surprise

new_target_codes = [1, 2, 3, 4, 5, 6]


data_frame = pd.DataFrame([["flow", "spatial",  "emotion", "code"]])
data_frame.to_csv(training_data_file, mode='w', header=False, index = False, sep = ',')
data_frame.to_csv(validation_data_file, mode='w', header=False, index = False, sep = ',')

def write_to_file(seg, valid): 
    data_frame =  pd.DataFrame([[seg[0].replace(baum_dir, ''), seg[1].replace(baum_dir, ''),  seg[2], seg[3]]]) #convert absolute path to relative path 
    if valid:
        data_frame.to_csv(validation_data_file, mode='a', header=False, index = False, sep = ',')
    else: 
        data_frame.to_csv(training_data_file, mode='a', header=False, index = False, sep = ',')
  
val = 8 # 7 train to 1 valid
last_folder = "" 
count = 0
for row in data:
    update_progress(float(count)/len(data))
    count += 1
    file_name = row[0]
    #print(file_name)
    folder = file_name.split('\\')[-3]
    row[3] = new_target_codes[target_codes.index(row[3])]
    frame = [row[0], row[1], row[2], row[3]]
 
    if ( folder != last_folder ):
        val -= 1
        if val == 0:
            valid = 1
            write_to_file(frame, 1)
            val = 8
        else:
            valid = 0
            write_to_file(frame, 0)
    elif valid:
        write_to_file(frame, 1)
    elif not valid:
        write_to_file(frame, 0)
    last_folder = folder
    
print ("Done", "\nTraining File: ",training_data_file , "\nValidation File: ", validation_data_file)

Progress: [############################################################] 100.0%
Done 
Training File:  ..\BAUM\training_data_1s.csv 
Validation File:  ..\BAUM\validation_data_1s.csv
